In [1]:
from tika import parser
import pandas as pd
from io import StringIO
import re

Desde el 32 al 34 > Únicamente texto

In [2]:
raw = parser.from_file('../data/pdf_data/Actualizacion_32_COVID-19.pdf')

In [3]:
ind_ini = raw['content'].find('en España:')
ind_fin = raw['content'].find('mundo:')

In [4]:
print(raw['content'][ind_ini:ind_fin])

en España: 
 
En esta semana se han detectado un total de 14 casos en España. Once de ellos tienen relación 
epidemiológica con el brote de Italia, y 3 permanecen en investigación.  
 
Hasta el momento se han registrado 16 casos: 6 en las Islas Canarias, 2 en la Comunidad Valenciana, 2 en 
Cataluña, 4 en Madrid, 1 en Andalucía y 1 en Baleares.  
 
Situación en el 


35 diferente

In [6]:
raw = parser.from_file('../data/pdf_data/Actualizacion_37_COVID-19.pdf')

fecha

In [7]:
def get_fecha(string):
    ind_ini = string.find('(COVID-19)')
    ind_fin = string.find('2020')
    return string[ind_ini:ind_fin].split('\n')[-1]+'2020'

In [8]:
fecha = get_fecha(raw['content'])

In [9]:
def parsing_table(string):
    first_number = re.search(r'[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?', string).group()
    new_lst = string.split(' ')
    ind = new_lst.index(first_number)
    final_list = [''.join(new_lst[:ind])]+new_lst[ind:]
    return [el for el in final_list if el != '']

In [35]:
def get_lst(string):
    ind_ini = string.lower().find('andalucía')
    ind_fin = min(string[ind_ini:].lower().find('total'), string[ind_ini:].lower().find('españa'))
    list_tab = string[ind_ini:ind_ini+ind_fin].split('\n')
    list_tab = [el.replace('.','') for el in [el.rstrip() for el in list_tab] if el != '']
    list_tab = [el.replace(',','.') for el in [el.rstrip() for el in list_tab] if el != '']
    return [parsing_table(el) for el in list_tab]

In [36]:
def save_csv(lst, path):
    if len(lst[0])==7:
        cols = ['CCAA', 'casos', 'IA','Hospitalizados', 'UCI', 'muertes', 'nuevos']
    elif len(lst[0])==5:
        cols = ['CCAA', 'casos', 'IA', 'UCI', 'muertes']
    elif len(lst[0])==4:
        cols = ['CCAA', 'casos', 'IA', 'UCI']
    elif len(lst[0])==3:
        cols = ['CCAA', 'casos', 'IA']
    data = pd.DataFrame(lst, columns = cols)
    data['fecha'] = [fecha]*data.shape[0]
    cols.insert(1, 'fecha')
    data[cols].to_csv(path, index=False)

In [37]:
raw = parser.from_file('../data/pdf_data/Actualizacion_51_COVID-19.pdf')
fecha = get_fecha(raw['content'])
lst = get_lst(raw['content'])
#save_csv(lst,'../data/csv_data/COVID_es_{}.csv'.format(fecha.replace('.','_')))

In [39]:
cols = ['CCAA', 'casos', 'IA','Hospitalizados', 'UCI', 'muertes', 'nuevos']

In [40]:
data = pd.DataFrame(lst, columns = cols)

In [42]:
data['fecha'] = [fecha]*data.shape[0]
cols.insert(1, 'fecha')

In [43]:
data[cols]

,CCAA,fecha,casos,IA,Hospitalizados,UCI,muertes,nuevos
0,Andalucía,21.03.2020,1515,17.68,605,53,40,228
1,Aragón,21.03.2020,424,31.30,192,33,22,64
2,Asturias,21.03.2020,486,47.03,133,12,5,142
3,Baleares,21.03.2020,246,20.88,60,14,4,43
4,Canarias,21.03.2020,348,15.65,139,29,7,61
5,Cantabria,21.03.2020,215,35.28,92,9,2,71
6,CastillaLaMancha,21.03.2020,1423,69.26,635,65,84,379
7,CastillayLeón,21.03.2020,1466,60.51,575,85,55,319
8,Cataluña,21.03.2020,4203,54.45,1681,298,122,933
9,Ceuta,21.03.2020,5,5.90,0,0,0,0
